# Forecasting Airline Passengers Using Multiple ML Models

This notebook demonstrates forecasting using the following models:
- Linear Regression
- Decision Tree
- KNN
- Naive Bayes
- Artificial Neural Network (ANN)

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

# تحميل البيانات
df = pd.read_csv("data/original data/airline-passengers.csv")
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['total_passengers']])

# إعداد بيانات تأخير (lag)
def create_lagged_features(data, lag=3):
    X, y = [], []
    for i in range(lag, len(data)):
        X.append(data[i-lag:i])
        y.append(data[i])
    return np.array(X), np.array(y)

X, y = create_lagged_features(scaled_data.flatten(), lag=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# تدريب النماذج
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=0),
    'KNN': KNeighborsRegressor(n_neighbors=3),
    'Naive Bayes': GaussianNB(),
    'ANN': MLPRegressor(hidden_layer_sizes=(50,), max_iter=1000, random_state=0)
}

predictions = {}

for name, model in models.items():
    if name == 'Naive Bayes':
        y_train_class = (y_train * 10).astype(int)
        model.fit(X_train, y_train_class)
        preds = model.predict(X_test) / 10
    else:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
    predictions[name] = scaler.inverse_transform(preds.reshape(-1, 1)).flatten()

# عرض النتائج
for name, pred in predictions.items():
    plt.figure(figsize=(10, 4))
    plt.plot(pred, label='Predicted')
    plt.title(f'{name} Forecast')
    plt.xlabel('Time Step')
    plt.ylabel('Total Passengers')
    plt.legend()
    plt.grid(True)
    plt.show()


# SVM
svm = SVR()
svm.fit(X_train, y_train)
predictions['SVM'] = scaler.inverse_transform(svm.predict(X_test).reshape(-1, 1)).flatten()

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)
predictions['Random Forest'] = scaler.inverse_transform(rf.predict(X_test).reshape(-1, 1)).flatten()
